In [1]:
import numpy as np
import pandas as pd
import torch
from time import time
from data import TargetData

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
test_data = pd.read_csv("evaluations/nfcf/test_userPages.csv")  # consists of random sample
test_data.rename(columns={'user_id': 'uid', 'like_id': 'mid'}, inplace=True)
test_data['rating'] = 1

test_data

,uid,mid,rating
0,1,425,1
1,4,48,1
2,4,1337,1
3,7,313,1
4,8,490,1
...,...,...,...
9943,6038,1616,1
9944,6039,990,1
9945,6039,1572,1
9946,6039,3050,1


In [3]:
data = TargetData()  # consists of latest / last item for each user

data.test

,uid,mid,rating
25,0,25,1
66,1,66,1
232,2,207,1
235,3,208,1
258,4,222,1
...,...,...,...
998205,6035,1048,1
998959,6036,294,1
999133,6037,1528,1
999166,6038,420,1


In [4]:
from models import NCF

ncf = NCF(6040, 3416, 128, np.array([128, 64, 32, 16]), 1).to(device)
ncf.load_state_dict(torch.load("saved_models/preTrained_NCF",  map_location=torch.device(device)))
ncf.to(device)

NCF(
  (user_emb): Embedding(6040, 128)
  (like_emb): Embedding(3416, 128)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (relu4): ReLU()
  (outLayer): Linear(in_features=16, out_features=1, bias=True)
  (out_act): Sigmoid()
)

In [5]:
from evaluators import eval_model

t1 = time()
hr2, ndcg2 = eval_model(ncf, data, num_users= data.num_users, device=device)
t2 = time()

print(f'{int(t2 - t1)} seconds')
print(f'hr: {round(hr2, 2)}')

4 seconds
hr: 1.0


In [14]:
from evaluators import evaluate_model

t1 = time()
avg_HR_preTrain, avg_NDCG_preTrain = evaluate_model(
    ncf,
    test_data.values,
    top_k=10,
    random_samples=100,
    num_items=3416,
    device=device
)
t2 = time()

print(f'{int(t2 - t1)} seconds')
print(f'hr: {round(avg_HR_preTrain[-1], 2)}')

14 seconds
hr: 1.0


In [9]:
from dump.evaluate import Evaluate

evaluator = Evaluate(data=data, testing_data=test_data, device=device)

t1 = time()
hr, ndcg = evaluator(model=ncf)
t2 = time()

print(f'{int(t2 - t1)} seconds')
print(f'hr: {round(hr, 2)}')

51 seconds
hr: 1.0
